## 0. Setup

In [ ]:
!pip install langchain langchain-openai langchain-community faiss-cpu pypdf chromadb tiktoken

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 999, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
^C


In [1]:
# Importaciones
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
import pickle

In [2]:
from google.colab import userdata
api_key = userdata.get('OPEN_AI')

## 1. Extracting files

In [ ]:
import os
import glob

# 🔹 1. Clonar el repo (cambia la URL por la de tu repo)
!git clone https://github.com/Jorhuer/Modulo1_Reto_RAG.git

# 🔹 2. Detectar automáticamente la carpeta del repo
# (asume que solo clonaste un repo dentro de /content)
repo_dirs = glob.glob("/content/*/")
repo_path = max(repo_dirs, key=os.path.getmtime).rstrip("/")  # el más reciente clonado
print(f"📂 Repo detectado en: {repo_path}")

# 🔹 3. Buscar el archivo PDF dentro de /datos
pdf_path = os.path.join(repo_path, "datos", "K32L2B3xRM.pdf")

if os.path.exists(pdf_path):
    print(f"✅ Archivo encontrado: {pdf_path}")
else:
    print(f"❌ No se encontró el archivo en: {pdf_path}")
    print("👉 Verifica que exista la carpeta 'datos' y que el PDF tenga exactamente ese nombre.")


Saving K32L2B3xRM.pdf to K32L2B3xRM (2).pdf
✅ Archivo subido: K32L2B3xRM (2).pdf (5058698 bytes)
📄 Rutas: /content/K32L2B3xRM (2).pdf


In [5]:
# Cargar el PDF
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Mostrar info básica
print(f"Número de páginas: {len(documents)}")
print(f"Texto de la primera página: {documents[0].page_content[:200]}...")

# Opcional: Guardar documentos crudos para inspección
with open("raw_documents.pkl", "wb") as f:
    pickle.dump(documents, f)

Número de páginas: 977
Texto de la primera página: K32 L2B Sub-Family Reference
Manual
Supports: K32L2B31VLH0A, K32L2B31VMP0A, K32L2B31VFT0A,
K32L2B31VFM0A, K32L2B21VLH0A, K32L2B21VMP0A,
K32L2B21VFT0A, K32L2B21VFM0A, K32L2B11VLH0A,
K32L2B11VMP0A, K32L...


## 2. Text Splitting into Chunks

In [6]:
# Configuración del splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamaño máximo por chunk
    chunk_overlap=200,  # Overlap para contexto
    length_function=len,  # Función para medir longitud
    separators=["\n\n", "\n", " ", ""]  # Jerarquía de splits
)

# Dividir documentos
chunks = text_splitter.split_documents(documents)

# Mostrar info
print(f"Número de chunks: {len(chunks)}")
print(f"Longitud del primer chunk: {len(chunks[0].page_content)} caracteres")
print(f"Primer chunk: {chunks[0].page_content[:150]}...")

Número de chunks: 2406
Longitud del primer chunk: 268 caracteres
Primer chunk: K32 L2B Sub-Family Reference
Manual
Supports: K32L2B31VLH0A, K32L2B31VMP0A, K32L2B31VFT0A,
K32L2B31VFM0A, K32L2B21VLH0A, K32L2B21VMP0A,
K32L2B21VFT0A,...


## 3. Embedding

In [ ]:
# Inicializar embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)

# Ejemplo: Embed un chunk de prueba
sample_embedding = embeddings.embed_query(chunks[0].page_content[:100])
print(f"Forma del embedding: {len(sample_embedding)} dimensiones")
print(f"Primeros 5 valores: {sample_embedding[:5]}")

Forma del embedding: 1536 dimensiones
Primeros 5 valores: [0.017576045199562206, 0.004715609571925872, 0.0039780309313411834, -0.007375786871508193, -0.020568308395154555]


## 4. Vector Stores

In [10]:
# Con Chroma
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
vectorstore.persist()

/tmp/ipython-input-2973032204.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## 5. Retriving from the Persistant Vector Datastore

In [15]:
# Cargar vector store persistente con Chroma
loaded_vectorstore = Chroma(
    persist_directory="chroma_db",  # Cambia "faiss_index" por "chroma_db"
    embedding_function=embeddings
)

## 6. Retrivers in Langchain

In [17]:
from langchain_openai import OpenAI

# Inicializar LLM (OpenAI)
llm = OpenAI(temperature=0.7, openai_api_key=api_key)

# Crear retriever básico
retriever = loaded_vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# Opcional: Retriever con compresión (reduce ruido)
# compressor = LLMChainExtractor.from_llm(llm)
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor, base_retriever=retriever
# )

# Cadena RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 'stuff' mete todos los docs en el prompt
    retriever=retriever,
    return_source_documents=True
)

# 7. Request

In [19]:
def ask_rag(question):
    result = qa_chain({"query": question})
    return result["result"]

# Ejemplo
print(ask_rag("\n¿Cuantos pines tiene el K32L2B3 por numero de parte?"))


El K32L2B3 tiene 48 pines en la versión QFN y 64 pines en la versión LQFP y MAPBGA. El número de parte no determina la cantidad de pines, ya que hay varias variantes del K32L2B3 con diferentes cantidades de memoria y funcionalidades. 


In [20]:
def ask_rag(question):
    result = qa_chain({"query": question})
    return result["result"]

# Ejemplo
print(ask_rag("\n¿Cuatos perifericos ADC soporta el K32L2B3?"))


El K32L2B3 soporta un total de 16 periféricos ADC. 


In [21]:
def ask_rag(question):
    result = qa_chain({"query": question})
    return result["result"]

# Ejemplo
print(ask_rag("\n¿Cuanta RAM tiee el K32L2B3?"))


El K32L2B3 tiene una opción de memoria de hasta 32 KB de RAM. 


In [23]:
def ask_rag(question):
    result = qa_chain({"query": question})
    return result["result"]

# Ejemplo
print(ask_rag("\n¿Cuanta ROM tiene el K32L2B3?"))

 16 KB of ROM.


In [24]:
def ask_rag(question):
    result = qa_chain({"query": question})
    return result["result"]

# Ejemplo
print(ask_rag("\n¿Como se implementa el modo de bajo consumo en el K32L2B3?"))


El modo de bajo consumo en el K32L2B3 se implementa a través del apagado de todos los relojes del sistema y del bus, la desactivación de los generadores de reloj en el MCG y la configuración de los reguladores y interruptores de energía internos para cumplir con los objetivos de consumo de energía. Este dispositivo también admite los modos de ejecución (RUN) y de ejecución muy baja en energía (VLPR).
